In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
train_dir = '/content/drive/MyDrive/dataset2/train'
test_dir = '/content/drive/MyDrive/dataset2/validation'

img_size = (150, 150)
batch_size = 32

# Create ImageDataGenerator for training data and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Create ImageDataGenerator for testing data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

# Load and prepare testing data
test_data = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')


Found 1031 images belonging to 2 classes.
Found 258 images belonging to 2 classes.


In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Reduce the learning rate dynamically
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
history = model.fit(train_data, steps_per_epoch=train_data.samples // batch_size, epochs=3, validation_data=test_data,validation_steps=test_data.samples//batch_size)

Epoch 1/3
32/32 [==============================] - 181s 6s/step - loss: 0.6380 - accuracy: 0.5926 - val_loss: 0.3118 - val_accuracy: 0.8633
Epoch 2/3
32/32 [==============================] - 66s 2s/step - loss: 0.1269 - accuracy: 0.9540 - val_loss: 0.0172 - val_accuracy: 0.9922
Epoch 3/3
32/32 [==============================] - 64s 2s/step - loss: 0.0540 - accuracy: 0.9824 - val_loss: 0.0254 - val_accuracy: 0.9961


In [8]:
import pickle

with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)



# model.evaluate(test_data)

**Changing the Directory to saved Model Directory**

In [ ]:
cd /content/drive/MyDrive/Mini Project

/content/drive/MyDrive/Mini Project


**Change Permission to run ngrok**

In [ ]:
!chmod +x ngrok

**Adding authtoken of ngrok**

In [ ]:
!./ngrok authtoken 2GRsSvZq1eX8gBkC8vSDTeb6fpa_2Zrn9KB4ZLeVWeCbZLHE1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


**Start the ngrok server**

In [ ]:
import subprocess
import time

# Start ngrok
ngrok_process = subprocess.Popen(['./ngrok', 'http', '5000'])

import requests

# Sleep for a few seconds to allow ngrok to start
time.sleep(5)

# Get ngrok URL using ngrok API
ngrok_url = 'http://localhost:4040/api/tunnels'
response = requests.get(ngrok_url)
ngrok_data = response.json()

# Extract the ngrok URL
ngrok_url = ngrok_data['tunnels'][0]['public_url']
print("Ngrok URL:", ngrok_url)


Ngrok URL: https://acdb-35-239-68-162.ngrok-free.app


**Creating Flask Application**

In [ ]:
from flask import Flask, request, jsonify, render_template
from PIL import Image
import numpy as np
import io
import base64
from tensorflow.keras.models import load_model
from flask import Flask, request, render_template
from tensorflow.keras.models import load_model
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib.pyplot as plt
import io
import numpy as np

app = Flask(__name__)

# Load the model and compile it (Replace 'my_model.h5' with your model's path)
model = load_model('Fake_image_detection_model.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training data (Replace with your data)
# X_train, y_train = ...
# history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

history = model.fit(train_data, steps_per_epoch=train_data.samples // batch_size, epochs=3, validation_data=test_data,validation_steps=test_data.samples//batch_size)
# Assuming 'history' contains training history (loss and accuracy)

# Function to generate accuracy and loss graph
def generate_graph():
    epochs = range(len(history.history['accuracy']))
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].plot(epochs, history.history['accuracy'], label='Training Accuracy')
    ax[0].plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    ax[0].set_title('Training and Validation Accuracy')
    ax[0].legend()
    ax[1].plot(epochs, history.history['loss'], label='Training Loss')
    ax[1].plot(epochs, history.history['val_loss'], label='Validation Loss')
    ax[1].set_title('Training and Validation Loss')
    ax[1].legend()

    # Save the plot to a BytesIO object
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plt.close()

    return img

@app.route('/', methods=['GET', 'POST'])
def predict():
    def predict():
    if request.method == 'POST':
        file = request.files['file']
        img = Image.open(file.stream)
        img = img.resize((150, 150))
        img = np.array(img) / 255.0
        img = np.reshape(img, (1, 150, 150, 3))
        prediction = loaded_model.predict(img)
        if prediction[0][0] > 0.5:
            result = "Real Image"
            confidence_percentage = f"{prediction[0][0] * 100:.2f}%"
        else:
            result = "Fake Image"
            confidence_percentage = f"{100 - prediction[0][0] * 100:.2f}%"

        # Convert image to base64 to display in HTML
        img = Image.fromarray((img.reshape(150, 150, 3) * 255).astype(np.uint8))
        image = io.BytesIO()
        img.save(image, format='PNG')
        image.seek(0)
        encoded_img = base64.b64encode(image.getvalue()).decode('utf-8')
        return render_template('index.html', prediction=prediction, confidence=confidence_percentage, image=encoded_img)

@app.route('/model_summary')
def model_summary():
    summary = []
    model.summary(print_fn=lambda x: summary.append(x))
    return render_template('model_summary.html', summary=summary)

@app.route('/model_performance')
def model_performance():
    graph = generate_graph()
    return render_template('model_performance.html', graph=graph.read().encode('base64'))

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:57:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:57:06] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:57:06] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 191ms/step


INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:57:17] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:57:18] "GET /static/style.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 71ms/step


...

INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:58:43] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:58:44] "GET /static/style.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:58:53] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:58:54] "GET /static/style.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 71ms/step


INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:59:03] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 15:59:04] "GET /static/style.css HTTP/1.1" 304 -
